In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [ ]:
# Old Static File
# df = pd.read_pickle("/Users/jenniferdimaano/Desktop/GitHub/rescue-chicago/petfinder-data/data/chicago_il_animals_cleaned.pkl")

# New Heroku DB Connection
# https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html
# EXAMPLE: df = pd.read_sql('test_data', 'postgres:///db_name') 
HEROKU_URL = os.getenv('HEROKU_POSTGRESQL_AMBER_URL')

uri = HEROKU_URL 
if uri.startswith("postgres://"):
    uri = uri.replace("postgres://", "postgresql://", 1)
df = pd.read_sql('petfinder_with_dates', uri)  


In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
# head = df.head()

In [ ]:
# cols = df.columns

In [ ]:
df.info()

In [ ]:
desc = df.describe(include='all')
print(desc)

In [ ]:
# # who is null?
null = df.isnull().sum().sort_values(ascending = False)
print(null)


In [ ]:
# dropping features with too many nulls
df = df.drop(columns=["id", "organization_id", "attribute_declawed", "attribute_declawed", "color_tertiary", 
                      "good_with_cats",
                      "good_with_children",
                      "good_with_dogs",
                      "breed_secondary",
                      "color_secondary"])

In [ ]:
# # What rows are missing coat number info?
# no_coat = df.loc[df['coat'].isnull()]
# # no_coat.head()

In [ ]:
# # Let's see what the distribution of passenger class is like for these people!
# nocoat_breed = no_coat['breed_primary'].value_counts()

In [ ]:
# breedprimary = df['breed_primary'].value_counts()['Labrador Retriever']

In [ ]:
# unique = df['coat'].unique()

In [ ]:
def fill_nan_mode(df, reference_column, feature):
    # Calculate the mode coat for each breed_primary
    mode_by_breed = df.groupby(reference_column)[feature].apply(lambda x: x.mode().iloc[0] if not x.isnull().all() else None)

    # Create a dictionary mapping each breed to its mode coat
    mode_dict = dict(mode_by_breed)

    # Fill the NaN values in 'coat' based on the breed using the mode_dict
    df[feature] = df.apply(lambda row: mode_dict[row[reference_column]] if pd.isna(row[feature]) and row[reference_column] in mode_dict else row[feature], axis=1)

    return df

In [ ]:

fill_nan_mode(df, 'breed_primary', 'coat')
fill_nan_mode(df, 'breed_primary', 'color_primary')

In [ ]:
df['breed_mixed'].unique()

In [ ]:
# df['coat'].isnull().sum()

In [ ]:
# df['color_primary'].isnull().sum()

In [ ]:
# df[df['color_primary'].isnull()]

In [ ]:
# df[df['color_primary'].isnull()]['breed_primary'].unique()

In [ ]:
# df['breed_primary'].value_counts()['Belgian Shepherd / Laekenois']

In [ ]:
# df[df['coat'].isnull()]['breed_primary'].unique()

In [ ]:
# df[df['coat'].isnull()]

In [ ]:
# Dropping rows with null coat and color primary
def drop_null_rows(df, feature):
    df.dropna(subset=[feature], inplace=True)
    return df

In [ ]:
drop_null_rows(df, 'coat')
drop_null_rows(df, 'color_primary')

In [ ]:
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()


# Create a scatter plot of numeric features against "los"
for col in num_cols:
    plt.scatter(df[col], df["los"])
    plt.xlabel(col)
    plt.ylabel("los")
    plt.show()


In [ ]:
# Bar plot of gender distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='gender', data=df)
plt.title('Gender Distribution')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()


In [ ]:
#pd.set_option('display.max_rows', None)
#df[df['gender'] == 'Unknown']
df.drop(df[df['gender'] == 'Unknown'].index, inplace=True)


In [ ]:
# Histogram of age distribution
plt.figure(figsize=(8, 6))
sns.histplot(df['age'])
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

In [ ]:
# Calculate the mean 'los' for each age category
mean_los_by_age_category = df.groupby('age')['los'].mean().reset_index().sort_values(by='los', ascending=False)

# Bar chart of mean 'los' by age category
plt.figure(figsize=(8, 6))
sns.barplot(data=mean_los_by_age_category, x='age', y='los')
plt.title('Mean LOS by Age Category')
plt.xlabel('Age Category')
plt.ylabel('Mean LOS')
plt.show()


In [ ]:
# List of variable names to create box plots for
variables = ['size', 'age', 'gender', 'attribute_spayed_neutered', 'attribute_house_trained']

# Create box plots for each variable
for variable in variables:
    plt.figure(figsize=(10, 8))
    
    if variable == 'breed_mix':
        # Handle 'breed_mix' separately
        unique_categories = df['breed_mix'].unique()
        for category in unique_categories:
            subset = df[df['breed_mix'] == category]
            sns.boxplot(x=variable, y='los', data=subset)
        
        plt.xlabel(variable.capitalize())
    elif variable in ['attribute_spayed_neutered', 'attribute_house_trained']:
        # Handle boolean features
        sns.boxplot(x=variable, y='los', data=df, order=[True, False])
        plt.xlabel(variable.capitalize())
    else:
        # Handle object features
        sns.boxplot(x=variable, y='los', data=df)
        plt.xlabel(variable.capitalize())
    
    plt.title(f'Length of Stay by {variable.capitalize()}')
    plt.ylabel('Length of Stay')
    plt.show()


In [ ]:
pd.set_option('display.max_rows', None)
df[df['los'] < 0]
df.drop(df[df['los'] < 0].index, inplace=True)

df.head()

In [ ]:
(df['los']<=0).sum()

In [ ]:
from scipy import stats

# Perform Box-Cox transformation
transformed_los, lambda_value = stats.boxcox(df['los'])

# Perform Box-Cox transformation
transformed_los, lambda_value = stats.boxcox(df['los'])

# Print the lambda value (to be used for inverse transformation if needed)
print("Lambda value:", lambda_value)

# Print the transformed 'los' values
print("Transformed 'los':", transformed_los)

In [ ]:
# # Heatmap of missing values
# plt.figure(figsize=(10, 8))
# sns.heatmap(df.isnull(), cmap='viridis', cbar=False)
# plt.title('Missing Values')
# plt.show()